# Prev

## include

In [8]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")
include("../../src/run_event.jl")

RunFusionAggregates (generic function with 1 method)

## model

In [9]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

  0.000002 seconds (4 allocations: 144 bytes)


ModelSet(TimeModel(150000.0, 0.5, 100, 50), InputModel(0.8, "../../data/init/Sphere"), OutputModel("Test_1", ""))

# Run

## Some Founded Parameters

### Cubic

In [10]:
# # Complete (Oriola Contractile Model) -> Normal Vectors of distance between cells without area
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.162);
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.166);
# Par1, Par2 = Cubic(0.0055,2.0,4.5), ContractilePar(0.164);

## Fusion Agg

In [11]:
# Run Model
Par1 ,Par2 = Cubic(0.01,2.0,3.0), ContractilePar(0.12)
# ContractilePar(0.02);
# RunFusionAggregates(model::ModelSet, Par1, Par2, 15)
size_agg = 15

15

In [12]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(size_agg).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

Run One Aggregate     ... 100%|██████████████████████████| Time: 0:02:42:54


In [13]:
run_test(agg, model,"Run One Aggregate     ")

Run One Aggregate     ... 100%|██████████████████████████| Time: 0:02:42


In [14]:
position=agg.Position
open("init_stable.xyz", "a") do f
    write(f, "Initial Stable\n")
    write(f, "t=0\n")
    writedlm(f,hcat(agg.Geometry.outline,Matrix(position)), ' ')
end

# Run fusion of two aggregates
agg = nothing
agg = FusionAggregate(
    [AggType("HEK_1", InteractionPar(Par1, Par2),position)], 
    model
)
run_test(agg, model, "Fusion of Two Aggregates")

display(sum(isnan.(agg.Simulation.Force.F), dims=1))
display(sum(agg.Simulation.Force.F .> 50, dims=1))
display(agg.Simulation.Force.F)
display(agg.Position)

Fusion of Two Aggregates... 100%|████████████████████████| Time: 0:05:45


1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

1×3 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.0402544   -0.0794487   0.0688348
  0.030842     0.0217593  -0.0991114
 -0.0120552   -0.0562472  -0.0353962
 -0.0342636   -0.082944    0.132142
  0.167617     0.265117    0.13594
  0.0408767   -0.122035   -0.00864512
  0.225646    -0.099355   -0.0212969
  0.108827    -0.0558468  -0.223999
 -0.0632429   -0.0834896  -0.0163379
 -0.0229806   -0.0272399   0.0385806
  0.0128079    0.104823    0.0618913
 -0.117317    -0.0532653  -0.150272
  0.0998449   -0.0120438   0.100937
  ⋮                       
 -0.175849     0.123752   -0.112467
  0.0843545   -0.305877   -0.0833149
 -0.0622022    0.0693202   0.0189047
  0.0338598    0.159158   -0.15333
 -0.00112858  -0.0656132   0.365704
  0.230988    -0.03882    -0.0844054
 -0.0753391   -0.185355    0.155496
  0.0154158   -0.0253524   0.149143
  0.0636786   -0.174231    0.161585
 -0.0168015   -0.0119065  -0.000515901
 -0.168425     0.169213    0.142679
  0.0221374    0.112909   -0.0210697

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  -6.75213    -3.08786    -7.26817
  -9.35362    -6.54993   -14.5194
 -11.145      -5.63674    -9.15235
 -11.7996      4.60285   -11.9538
  -2.50078    -1.94459    -6.09248
  -0.844429   -5.99576    -7.05821
  -3.03196    -9.42514    -9.58259
   2.33449     2.59131   -12.3456
  -9.58258    -5.51144   -15.1318
 -17.1362     -2.03378    -5.57148
   0.50939     0.663519  -17.5924
  -6.28448    -1.705     -16.2597
 -10.7489     -3.59949   -12.8119
   ⋮                     
 -14.6829      2.84559   -12.3109
   1.11389    -1.05094   -11.3391
  -4.44213    -9.25932    -6.72667
  -8.28064   -10.9193    -10.4528
  10.1866      3.16917    -6.55711
   1.77957     0.400614   -3.95176
  -2.40036     4.2887     -3.02979
  -4.12884    -0.562859  -16.4643
  -1.75811    -8.7731    -13.7171
  11.0477     -1.20158    -4.97205
   5.87946     6.63956   -14.1216
  -8.5056     -3.64359   -17.2102